# Radiotelescope - HI observation

In [ ]:
# Mostra todos os resultados da célula, não apenas o último
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"
# Truque para consertar caminhos do jupyter: variável __file__ não é definida em IPYTHON.
import project_path
import logging
import os
import sys
from matplotlib.colors import LogNorm
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.path.split(os.getcwd())[0])))
sys.path.append(os.path.abspath(os.path.join(os.path.split(os.getcwd())[0], "radiotelescope")))
sys.path.append(os.path.abspath(os.path.join(os.path.split(os.getcwd())[0], "radiotelescope/GNURadio")))
from radiotelescope.observations.observations import Observations as Obs
from radiotelescope.instruments import RTLSDRGNU
from radiotelescope.observations.observations import RFI_filter, plot_mosaic
logger = logging.getLogger("radiotelescope")
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

## Observações HI 

### Corneta Baixa

In [ ]:
RTLSDRGNU.GNUScript = "radiotelescope.GNURadio.GNUController"
RTLSDRGNU.name = "SDR_BAIXA"
RTLSDRGNU.controller.local_folder

In [ ]:
# SKY
scan_dict = {"rtlsdr": "rtl=0", 
             "name": RTLSDRGNU.controller.local_folder + RTLSDRGNU.name,  # prefixo para salvar os arquivos.
             "vec_length": 2048,  # resolução espectral.
             "samp_rate": 2000000,  # amostragem sem perda.
             "gain": 49.6,  # fixa AGC.
             "freq": 1420e6,  # Hidrogênio neutro
             "n_integration": 2000,  # 2s por espectro.
             "n_samples": 480,  # número de amostras por arquivo.
             "duration": 1000,  # ~15min
             "csv": False, 
             "fit": True
            }
scan_dict_SKY = {**scan_dict, "mode": "59"}
scan_dict_COLD = {**scan_dict, "mode": "01"}

Apontar a corneta para altitude baixa

In [ ]:
RTLSDRGNU.observe(**scan_dict_SKY)

Colocar terminação de $50 \Omega$

In [ ]:
RTLSDRGNU.observe(**scan_dict_COLD)

### Corneta Alta

In [ ]:
RTLSDRGNU.name = "SDR_ALTA"
scan_dict_SKY = {**scan_dict, "name": RTLSDRGNU.controller.local_folder + RTLSDRGNU.name}
scan_dict_COLD = {**scan_dict, "name": RTLSDRGNU.controller.local_folder + RTLSDRGNU.name}

Aponte a corneta para Altitude alta

In [ ]:
RTLSDRGNU.observe(**scan_dict_SKY)

Colocar terminação de $50 \Omega$

In [ ]:
RTLSDRGNU.observe(**scan_dict_COLD)

## Analisando Dados

#### Corneta Baixa

In [ ]:
obs = Obs(duration=pd.Timedelta(24, unit="h")).initialize()
obs.backend = RTLSDRGNU
RTLSDRGNU.name = "SDR_BAIXA"
obs.load_observation(extension="fit");
df_baixa = obs.data

In [ ]:
obs.plot_waterfall(obs.data, norm=LogNorm());

In [ ]:
df_cal, _ = obs.calibrate(data=obs.data, T_rx=300)
df_rfi, _, _ = RFI_filter(df_cal, threshold=3, window=100)

In [ ]:
obs.plot_waterfall(df_cal, norm=LogNorm());

In [ ]:
plot_mosaic([obs.data, df_cal, df_rfi], norm=LogNorm());

In [ ]:
obs.make_sky().sky.plot_timeseries(obs.data, interval="30s");

#### Corneta Alta

In [ ]:
obs = Obs(duration=pd.Timedelta(24, unit="h")).initialize()
obs.backend = RTLSDRGNU
RTLSDRGNU.name = "SDR_ALTA"
obs.load_observation(extension="fit");
df_alta = obs.data

In [ ]:
obs.plot_waterfall(obs.data, norm=LogNorm());

In [ ]:
df_cal, _ = obs.calibrate(data=obs.data, T_rx=300)
df_rfi, _, _ = RFI_filter(df_cal, threshold=3, window=100)

In [ ]:
obs.plot_waterfall(df_cal, norm=LogNorm());

In [ ]:
plot_mosaic([obs.data, df_cal, df_rfi], norm=LogNorm());

In [ ]:
obs.make_sky().sky.plot_timeseries(obs.data, interval="30s");

### Comparando Observações

In [ ]:
plot_mosaic([df_baixa, df_alta], norm=LogNorm());